Goal: we want to approximate the dominant eigenfunctions of the transfer operator underlying some observed dynamics.

To do this, we need to choose a good basis set. The variational principle of molecular kinetics lets us choose the optimal linear combinations of a given set of basis functions. In MSMs, these basis functions are indicator functions on a partitioning of the configuration space. In tICA, these basis functions are arbitrary input variables, usually internal coordinates like dihedral angles.

Another approach to constructing a basis set is to measure RMSD or weighted RMSD to some reference structures. A major benefit of that approach is that the model should now be differentiable with respect to the parameters defining the basis functions-- the locations of the references, and their associated weights.

In [1]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
%matplotlib inline
import mdtraj
plt.rc('font', family='serif')

In [2]:
# fetch example data

from msmbuilder.example_datasets import FsPeptide
dataset = FsPeptide().get()
fs_trajectories = dataset.trajectories
fs_t = fs_trajectories[0]

loading trajectory_1.xtc...
loading trajectory_10.xtc...
loading trajectory_11.xtc...
loading trajectory_12.xtc...
loading trajectory_13.xtc...
loading trajectory_14.xtc...
loading trajectory_15.xtc...
loading trajectory_16.xtc...
loading trajectory_17.xtc...
loading trajectory_18.xtc...
loading trajectory_19.xtc...
loading trajectory_2.xtc...
loading trajectory_20.xtc...
loading trajectory_21.xtc...
loading trajectory_22.xtc...
loading trajectory_23.xtc...
loading trajectory_24.xtc...
loading trajectory_25.xtc...
loading trajectory_26.xtc...
loading trajectory_27.xtc...
loading trajectory_28.xtc...
loading trajectory_3.xtc...
loading trajectory_4.xtc...
loading trajectory_5.xtc...
loading trajectory_6.xtc...
loading trajectory_7.xtc...
loading trajectory_8.xtc...
loading trajectory_9.xtc...


In [5]:
# 1. internal coordinate basis sets

from msmbuilder.featurizer import DihedralFeaturizer

basis_sets = dict()

dih_model=DihedralFeaturizer()
X = dih_model.fit_transform(fs_trajectories)
basis_sets['dihedral_phi_psi'] = X

dih_model=DihedralFeaturizer(types=['phi', 'psi', 'omega', 'chi1', 'chi2', 'chi3', 'chi4'])
X = dih_model.fit_transform(fs_trajectories)
basis_sets['dihedral_all'] = X

In [73]:
# 2. RMSD to reg-space refs
references = [x[::4000] for x in fs_trajectories]
refs = references[0]
for ref in references[1:]:
    refs = refs + ref


# oh, this is already built into MSMBuilder
#def compute_rmsds_to_refs(trajectories,refs):
#    basis_exps = []
#    for traj in trajectories:
#        rmsd_to_refs = np.zeros((len(traj),len(refs)))
#        for i in range(len(refs)):
#            rmsd_to_refs[:,i] = mdtraj.rmsd(traj,refs,i)
#        basis_exps.append(rmsd_to_refs)
#    return basis_exps
    
#basis_sets['rmsd_reg'] = compute_rmsds_to_refs(fs_trajectories,refs)

from msmbuilder.featurizer import RMSDFeaturizer
rmsdf = RMSDFeaturizer(refs)
basis_sets['rmsd_reg'] = rmsdf.fit_transform(fs_trajectories)

print(len(refs))

84


In [52]:
# 3. RMSD to cluster-center refs

# pick cluster centers by k-medoids
from msmbuilder.cluster import MiniBatchKMedoids
kmed = MiniBatchKMedoids(50,batch_size=200)
kmed.fit(X)

MiniBatchKMedoids(batch_size=200, max_iter=5, max_no_improvement=10,
         metric='euclidean', n_clusters=50, random_state=None)

In [59]:
# extract examplar configurations
clever_refs = []
for ind in kmed.cluster_ids_:
    clever_refs.append(fs_trajectories[ind[0]][ind[1]])

# convert list of length-1 mdtraj Trajectories to a single trajectory
# -- currently doing this in the most inefficient way possible, but it's 
# a small list so it doesn't matter too much
clever_ref = clever_refs[0]
for i in range(1,len(clever_refs)):
    clever_ref = clever_ref + clever_refs[i]
clever_refs = clever_ref
print(len(clever_refs))

50


In [61]:
basis_sets['rmsd_kmed'] = compute_rmsds_to_refs(fs_trajectories,clever_refs)

In [26]:
# 4. wRMSD to reg-space refs

from MDAnalysis.analysis.rms import rmsd as wRMSD

# compute weights from atomwise deviations
atomwise_deviations = np.load('fs_atomwise_deviations_tau=20.npy')
mean = atomwise_deviations.mean(0)
weights = np.exp(-mean/0.065)

In [84]:
def compute_wrmsd_traj(trajectories,refs,weights):
    ''' compute wRMSDs from trajectories to a list of references'''
    
    basis_exps = []
    for traj in trajectories:
        wrmsd_to_refs = np.zeros((len(traj),len(refs)))
        for i in range(len(traj)):
            for j in range(len(refs)):
                wrmsd_to_refs[i,j] = wRMSD(refs.xyz[j],traj.xyz[i],weights=weights,center='True')
        basis_exps.append(wrmsd_to_refs)
    return basis_exps

weighted_trajs = compute_wrmsd_traj(fs_trajectories,refs,weights)
basis_sets['wrmsd_exp'] = weighted_trajs

In [77]:
np.save('wrmsd_exp.npy',weighted_trajs)
#np.save('wrmsd_inv.npy',inv_mean_weighted_trajs)
#np.save('wrmsd_mean.npy',mean_weighted_trajs)

In [85]:
from msmbuilder.decomposition import tICA

def score(basis_set,m=10,lag_time=100):
    tica = tICA(lag_time=lag_time)
    tica.fit(basis_set)
    return np.sum(tica.eigenvalues_[:m])

In [88]:
scores = dict()
for name in basis_sets:
    scores[name] = score(basis_sets[name],m=20,lag_time=100)

In [89]:
scores.items()

[('dihedral_all', 17.84744612314757),
 ('rmsd', 13.109391916360241),
 ('dihedral_phi_psi', 17.706953927713055),
 ('rmsd_kmed', 9.0194490101577163),
 ('wrmsd_exp', 13.541230898294337),
 ('rmsd_reg', 13.109391916392024)]